In [237]:
class AnalysisHCI:

    def __init__(self, directory):
        # print('Make sure the directory has forward slashes')
        import os
        import pandas as pd
        self.directory = directory
        self.fileNameDF = self.FileNameDF()
        self.rawData = self.RawData()
        self.ProcessData()
        os.chdir(directory)
        
    def FileNameDF(self):
        import pandas as pd
        import numpy as np
        import glob
        self.fileNameDF = pd.DataFrame(columns=['file'])
        self.fileNameDF['file'] = glob.glob('participant_*')  
        self.fileNameDF.sort_values(by=['file'], inplace=True)
        return  self.fileNameDF
    def RawData(self):
        import pandas as pd
        self.rawData = pd.DataFrame()
        for file in self.fileNameDF['file']:
            f = open(file, "r")
            j = f.readline()
            f.close()
            df = pd.DataFrame(eval(f'[{j}]'))
            df['file'] = file
            self.rawData = pd.concat([self.rawData, df], axis=0)
        self.rawData.set_index('file', inplace=True)
        self.rawData['screenSize'] = self.rawData['scale'].map(lambda x: 38 if x == 0.2405 else 42 )
        return self.rawData
    def Fuzzy (self, targetPhrase,inputText):    
        from fuzzywuzzy import fuzz
        from fuzzywuzzy import process
        return fuzz.ratio(targetPhrase,inputText)
    def getCorrect (self, targetPhrase, inputText):
        count = 0
        stop = min(len(targetPhrase), len(inputText))
        for i, char in enumerate(targetPhrase):
            if i == stop:
                break
            if char == inputText[i]:
                count += 1
        return count  
    def getIF (self, targetPhrase,inputPhrase):
        import re
        inputPhrase = inputPhrase.replace('-delete-', '<')
        deletes = re.findall(r'<+',inputPhrase) # all the block of deletes that have been entered ('<<', '<', '<<<')
        m = re.findall(r'([\w\s]*)<+([\w\s]*)',inputPhrase) # the characters before and after the delete
        indexes = [ i.start() for i in re.finditer(r'<+',inputPhrase)] # the indexes of all the delete blocks
        count = 0 # count of all the incorrect fixes
        print(inputPhrase)
        print(targetPhrase)
        for i, s in enumerate(m):
            error, replace = s
            length = len(deletes[i])
#             priorDeletes = 0 if len(''.join(deletes[0:i])) == 0 else len(''.join(deletes[0:i])) + 1
            priorDeletes = len(''.join(deletes[0:i]))*2
            index = indexes[i] - length - priorDeletes
            replace = replace[:length]
            correctReplace = targetPhrase[index:index+length] 
#             print(f'replace: [{replace}]\tcorrect: [{correctReplace}]')
            count += self.getCorrect(correctReplace, replace)
            print(f'replace: [{replace}]\tcorrect: [{correctReplace}]')
        print(f'Count: {count}\n')
        return count

    def ProcessData (self):
        from fuzzywuzzy import fuzz
        from fuzzywuzzy import process
        fuzzyResult = []
        correctResult = []
        ifResult = []
        for f in self.rawData.index:
            file = self.rawData.loc[f]
            inputPhrase = file.inputPhrase
            inputText = file.inputText
            targetPhrase = file.targetPhrase
            fuzzyResult.append(fuzz.ratio(targetPhrase,inputText))
            correctResult.append(self.getCorrect(targetPhrase, inputText))
            ifResult.append(self.getIF(targetPhrase,inputPhrase))
        self.rawData['fuzzyRatio'] = fuzzyResult
        self.rawData['correct'] = correctResult
        self.rawData['IF'] = ifResult
        self.rawData['characters'] =  self.rawData['targetPhrase'].map(lambda x: len(x))
        self.rawData['INF'] = self.rawData['keyPressedTimes'] -  self.rawData['correct'] - 2* self.rawData['deleteTimes']
        self.rawData['accuracy/time/char'] = self.rawData['fuzzyRatio']/self.rawData['time']/self.rawData['characters']
        self.rawData['totalErrorRate'] = (self.rawData['INF'] + self.rawData['IF'])/(self.rawData['correct'] +self.rawData['INF'] + self.rawData['IF'])
        return self.rawData
    def totalErrorPivot (self):
        return self.rawData.pivot_table( index='id', columns= ["screenSize", 'type'], values='totalErrorRate')
        
    
     

In [238]:
directory = '/Users/jenny/Documents/School/IBBME/CSC2514/CSC2514-Assignments/HCI-Assignment2/Analysis/Data/ParticipantData'
a2 = AnalysisHCI(directory)

all good boyd<s f<ded<serve fudge
all good boys deserve fudge
replace: [s]	correct: [s]
replace: [d]	correct: [d]
replace: [s]	correct: [s]
Count: 3

obligations must bd <<e mey<t first
obligations must be met first
replace: [e ]	correct: [e ]
replace: [t]	correct: [t]
Count: 3

a<a picture is worth ,a<<,a< <<<m< mamy<<ny words
a picture is worth many words
replace: [a]	correct: [a]
replace: []	correct: [ma]
replace: [ ]	correct: [a]
replace: [m]	correct: [ ma]
replace: [ ]	correct: [ ]
replace: [ny]	correct: [ny]
Count: 4

p.<leade<<se keep this co n<<nfidential
please keep this confidential
replace: [l]	correct: [l]
replace: [se]	correct: [se]
replace: [nf]	correct: [nf]
Count: 5

what a monl<ket <<y sers<<es a mo m<<nkey wil.<<..<<ll dp<o
what a monkey sees a monkey will do
replace: [k]	correct: [k]
replace: [y ]	correct: [y ]
replace: [es]	correct: [es]
replace: [nk]	correct: [nk]
replace: []	correct: [ll]
replace: [ll]	correct: [ll]
replace: [o]	correct: [o]
Count: 10

motiba<<vat

In [239]:
a2.rawData

deleteTimes id  \
file                                         
participant_1_trial_1.txt             3  1   
participant_1_trial_10.txt            3  1   
participant_1_trial_11.txt           10  1   
participant_1_trial_12.txt            5  1   
participant_1_trial_13.txt           12  1   
participant_1_trial_14.txt            9  1   
participant_1_trial_15.txt            7  1   
participant_1_trial_16.txt            0  1   
participant_1_trial_17.txt            3  1   
participant_1_trial_18.txt            5  1   
participant_1_trial_19.txt            5  1   
participant_1_trial_2.txt             4  1   
participant_1_trial_20.txt            3  1   
participant_1_trial_21.txt            9  1   
participant_1_trial_22.txt            2  1   
participant_1_trial_23.txt            2  1   
participant_1_trial_24.txt           19  1   
participant_1_trial_25.txt            0  1   
participant_1_trial_26.txt            1  1   
participant_1_trial_27.txt            3  1   
participant_1_trial_28.txt            0  1   
participant_1_trial_29.txt            0  1   
participant_1_trial_3.txt             9  1   
participant_1_trial_30.txt            0  1   
participant_1_trial_31.txt            4  1   
participant_1_trial_32.txt            2  1   
participant_1_trial_33.txt            0  1   
participant_1_trial_34.txt            2  1   
participant_1_trial_35.txt            1  1   
participant_1_trial_36.txt            2  1   
...                                 ... ..   
participant_3_trial_26.txt            4  3   
participant_3_trial_27.txt           11  3   
participant_3_trial_28.txt            4  3   
participant_3_trial_29.txt            5  3   
participant_3_trial_3.txt             0  3   
participant_3_trial_30.txt            2  3   
participant_3_trial_31.txt            4  3   
participant_3_trial_32.txt            7  3   
participant_3_trial_33.txt            5  3   
participant_3_trial_34.txt            6  3   
participant_3_trial_35.txt            8  3   
participant_3_trial_36.txt            5  3   
participant_3_trial_37.txt            9  3   
participant_3_trial_38.txt            6  3   
participant_3_trial_39.txt            8  3   
participant_3_trial_4.txt             0  3   
participant_3_trial_40.txt            8  3   
participant_3_trial_41.txt            9  3   
participant_3_trial_42.txt            8  3   
participant_3_trial_43.txt            2  3   
participant_3_trial_44.txt            5  3   
participant_3_trial_45.txt           10  3   
participant_3_trial_46.txt            5  3   
participant_3_trial_47.txt           21  3   
participant_3_trial_48.txt            6  3   
participant_3_trial_5.txt             0  3   
participant_3_trial_6.txt             0  3   
participant_3_trial_7.txt             0  3   
participant_3_trial_8.txt             0  3   
participant_3_trial_9.txt             0  3   

                                                                  inputPhrase  \
file                                                                            
participant_1_trial_1.txt   all good boyd-delete-s f-delete-ded-delete-ser...   
participant_1_trial_10.txt  obligations must bd -delete--delete-e mey-dele...   
participant_1_trial_11.txt  a-delete-a picture is worth ,a-delete--delete-...   
participant_1_trial_12.txt  p.-delete-leade-delete--delete-se keep this co...   
participant_1_trial_13.txt  what a monl-delete-ket -delete--delete-y sers-...   
participant_1_trial_14.txt  motiba-delete--delete-vational semimz-delete--...   
participant_1_trial_15.txt  be persix-delete-stant to wu-delete-i n-delete...   
participant_1_trial_16.txt                        drove my chevy to the levee   
participant_1_trial_17.txt  the insulation ix-delete-s mo-delete--delete-n...   
participant_1_trial_18.txt  please take a bath tni-delete--delete-his mp-d...   
participant_1_trial_19.txt  he coi-delete-oled off after shd-delete-r-dele...   
participant_1_trial_2.txt   arguinb-delete-g with tb-delete-ne-delete--del..

In [243]:
a2.rawData['time'].mean()

29.502489583333332

In [244]:
a2.totalErrorPivot()

screenSize        38                  42          
type          normal      zoom    normal      zoom
id                                                
1           0.183525  0.036656  0.141517  0.042469
3           0.177559  0.036151  0.235188  0.002604

In [247]:
a2.rawData.pivot_table(index='id',
                       columns=["screenSize", 'type'], 
                       values=['keyPressedTimes', 'deleteTimes', 'time', 'totalErrorRate']
                      )

deleteTimes                               keyPressedTimes  \
screenSize          38                  42                        38   
type            normal      zoom    normal      zoom          normal   
id                                                                     
1             5.583333  1.250000  6.333333  1.416667       40.583333   
3             8.083333  1.333333  5.833333  0.083333       46.166667   

                                                  time                        \
screenSize                    42                    38                    42   
type             zoom     normal       zoom     normal       zoom     normal   
id                                                                             
1           33.083333  42.250000  33.166667  22.607400  31.983758  22.147858   
3           35.666667  41.416667  30.750000  27.670558  40.225783  26.505475   

                      totalErrorRate                                
screenSize                        38                  42            
type             zoom         normal      zoom    normal      zoom  
id                                                                  
1           30.155158       0.183525  0.036656  0.141517  0.042469  
3           34.723925       0.177559  0.036151  0.235188  0.002604

In [79]:
df.loc['participant_2_trial_5.txt'].targetPhrase

'mom made her a turtleneck'

In [104]:
inputPhrase = 'ad-delete-for your informatiohns only'
inputText = 'for your informatiohns only'
targetPhrase = 'for your information only'

file = df.loc['participant_1_trial_2.txt']
inputPhrase = file.inputPhrase
inputText = file.inputText
targetPhrase = file.targetPhrase

print(inputPhrase)
print(inputText)
print(targetPhrase)


def getCorrect (targetPhrase, inputText):
    count = 0
    stop = min(len(targetPhrase), len(inputText))
    for i, char in enumerate(targetPhrase):
        if i == stop:
            break
        if char == inputText[i]:
            count += 1
#         print(char, inputText[i])
    return count   
        
getCorrect(targetPhrase,inputText)        



no ec-delete-xchange wef-delete--delete--delete-without a bill
no exchange without a bill
no exchange without a bill


26

In [105]:
def getINF (targetPhrase,inputText):
    correct = getCorrect (targetPhrase, inputText)
    return len(targetPhrase) - correct

getINF (targetPhrase,inputText)

0

In [127]:
def getIF (targetPhrase,inputPhrase):
    import re
    inputPhrase = inputPhrase.replace('-delete-', '<')
    deletes = re.findall(r'<+',inputPhrase) # all the block of deletes that have been entered ('<<', '<', '<<<')
    m = re.findall(r'([\w\s]*)<+([\w\s]*)',inputPhrase) # the characters before and after the delete
    indexes = [ i.start() for i in re.finditer(r'<+',inputPhrase)] # the indexes of all the delete blocks
    count = 0 # count of all the incorrect fixes
    for i, s in enumerate(m):
        error, replace = s
        length = len(deletes[i])
        priorDeletes = ''.join(deletes[0:i])
        index = indexes[i] - length - len(priorDeletes) -1
        replace = replace[:length]
        correctReplace = targetPhrase[index:index+length] 
        print(f'replace: [{replace}]\tcorrect: [{correctReplace}]')
        count += getCorrect(correctReplace, replace)
    return count


print(inputPhrase)
print(inputText)
print(targetPhrase)

getIF (targetPhrase,inputPhrase)

no ec-delete-xchange wef-delete--delete--delete-without a bill
no exchange without a bill
no exchange without a bill
replace: [x]	correct: [e]
replace: [wit]	correct: [wit]


3

In [122]:
def fuzzy (targetPhrase,inputPhrase):
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    return fuzz.ratio(targetPhrase,inputText)

fuzzy (targetPhrase,inputPhrase)

100

In [248]:
a2.getIF('the quick brown','thw quick<<<<<<<e quick brown')

thw quick<<<<<<<e quick brown
the quick brown
replace: [e quick]	correct: [e quick]
Count: 7



7